### Deep Learning Miniproject - Audio

AVS 8th Semester - Group 841

#### Import packages

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import IPython.display as ipd

import torch
import torchaudio
from torch import nn
import torch.optim

from datasets import load_dataset, DatasetDict, load_metric
from transformers import ASTFeatureExtractor, ASTForAudioClassification, ASTConfig, TrainingArguments, Trainer

import wandb

/home/ubuntu/miniconda3/envs/dpl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Parameters

In [2]:
# data loading
AUDIO_DIR = "./data/"
CSV_DIR = "./data/metadata_compiled.csv"
FILE_TYPE = ".mp3"

# model
SAMPLING_RATE = 16000
BATCH_SIZE = 4
LEARNING_RATE = 5e-4
CHECKPOINT = 'MIT/ast-finetuned-audioset-10-10-0.4593'

MAX_DURATION = 1
NUM_CLASSES = 3
HIDDEN_LAYER_SIZE = 768
NUM_HIDDEN_LAYERS = 12
MAX_SEQ_LENGTH = MAX_DURATION * SAMPLING_RATE

In [3]:
#.csv file loading
df = pd.read_csv(CSV_DIR)

#### 1. Explore the dataset through code

a. How many samples does the dataset contain?

In [ ]:
#Check no. samples
print(f'Number of samples : {df.shape[0]}')

b. How many classes? How many samples per class? Show a histogram of the number of intances per class

In [ ]:
print(f'Number of classes: {len(df["status"].unique())}.\n\
    Classes: {df["status"].unique()}\n\
    {pd.value_counts(df["status"], dropna=False)}')

In [ ]:
pd.value_counts(df['status'], dropna=False).plot.bar()
plt.show()

c. Play a random sample from each class

In [ ]:
# playing healthy
healthy = df[df['status'] == 'healthy'].sample()['uuid'].item()
path = AUDIO_DIR + healthy + FILE_TYPE
print(path)
y, sr = torchaudio.load(path)
ipd.Audio(y, rate=sr)

In [ ]:
# playing COVID-19
covid = df[df['status'] == 'COVID-19'].sample()['uuid'].item()
path = AUDIO_DIR + covid + FILE_TYPE
y, sr = torchaudio.load(path)
ipd.Audio(y, rate=sr)

In [ ]:
# playing symptomatic
symptomatic = df[df['status'] == 'symptomatic'].sample()['uuid'].item()
path =  AUDIO_DIR + symptomatic + FILE_TYPE
y, sr = torchaudio.load(path)
ipd.Audio(y, rate=sr)

d. Describe if/how you think the data distribution will affect training of a classifier

e. Decide what part of the dataset to use; all, some classes, some samples. Motivate your choice

#### 2. Use a neural network of your own chose to classify the dataset. Explain your choice and at least one alternative. Document your experiences.

#### Audio Spectogram Transformer Implementation

#### Create Dataset

Load soundfiles into dataset

In [4]:
# must have metadata.csv with 'file_name' column to have also the features
# unsplitted dataset
dataset = load_dataset("audiofolder", data_dir=AUDIO_DIR, split="train")

Resolving data files: 100%|██████████| 16226/16226 [00:00<00:00, 20058.77it/s] 
Found cached dataset audiofolder (/home/ubuntu/.cache/huggingface/datasets/audiofolder/default-ca7edb5d3275be7e/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc)


Split up the data : `training` - 70%, `validation` - 20%, `test` - 10%

In [ ]:
train_testvalid = dataset.train_test_split(test_size=0.3)
test_valid = train_testvalid['test'].train_test_split(test_size=1/3)

dataset = DatasetDict({
    'train' : train_testvalid['train'],
    'test' : test_valid['test'],
    'valid' : test_valid['train']
})

List the labels from DataFrame

In [ ]:
# getting the labels for classification
labels = list(df["status"].unique()[1:])

label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

num_classes = len(labels)

##### Model and Feature Extractor

Run this cell if you want to use `random weights`

In [ ]:
# use this for [RANDOM WEIGHTS] - no pretraining
config = ASTConfig(
    hidden_size=768, # default : 768
    num_hidden_layers=12, # default : 12
    hidden_dropout_prob= 0.0 # def.: 0.0
    attention_probs_dropout_prob=0.0 # def.: 0.0
)

# basically tokenizer
# input normalization: mean = 0, std = 0.5
feature_extractor = ASTFeatureExtractor(config, sampling_rate=SAMPLING_RATE, num_mel_bins=32, mean=0, std=0.5)

model = ASTForAudioClassification(config)
# weights must be the same for the model and the tokenizer/feature extractor 

Run this cell if you want to use `pre-trained weights`

In [ ]:
# feature extractor and model
feature_extractor = ASTFeatureExtractor(
    CHECKPOINT
)

model = ASTForAudioClassification.from_pretrained(
    CHECKPOINT, 
    num_labels=len(labels),
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
    )

#### Tokenizing dataset

Function for `map`

In [ ]:
def preprocess(examples):
    audio_arr = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arr,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=MAX_SEQ_LENGTH,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )
    
    label = [int(label2id[x]) for x in examples["label"]]
    
    inputs["label"] = label
    return inputs

Tokenizing the dataset

In [ ]:
dataset['train'] = dataset['train'].map(preprocess, remove_columns=["audio"], batched=True)
dataset['valid'] = dataset['valid'].map(preprocess, remove_columns=["audio"], batched=True)
dataset['test']= dataset['test'].map(preprocess, remove_columns=["audio"], batched=True)

### Training the model

Set `optimizer` and `scheduler`

In [ ]:
# setting the optimizer and the scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

Metric computing function : calculates `accuracy`, `precision`, `recall` and `f1 score`

In [ ]:
def compute_metrics(eval_pred):
    metrics = dict()

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy_metric = load_metric('accuracy')
    precision_metric = load_metric('precision')
    recall_metric = load_metric('recall')
    f1_metric = load_metric('f1')

    metrics.update(accuracy_metric.compute(predictions=predictions, references=labels))
    metrics.update(precision_metric.compute(predictions=predictions, references=labels, average='weighted'))
    metrics.update(recall_metric.compute(predictions=predictions, references=labels, average='weighted'))
    metrics.update(f1_metric.compute(predictions=predictions, references=labels, average='weighted'))

    return metrics

Setting up `wandb` to visualize training results

In [ ]:
# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="cough-project"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

Defining `TrainingArguments` and `Trainer`

In [ ]:
# the hyperparams for Trainer
training_arg = TrainingArguments(
    output_dir="output",
    report_to="wandb",
    learning_rate=LEARNING_RATE,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    #num_train_epochs= max_epochs,S
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    max_steps=1000, 
    logging_steps=50,
    eval_steps=200, 
    eval_accumulation_steps=1, 
    load_best_model_at_end=True,
    warmup_steps=50,
    save_total_limit=2,
    #metric_for_best_model='accuracy'
    )

# defining trainer
trainer = Trainer(
    model=model,
    args=training_arg,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler)
)

Do the training

In [ ]:
trainer.train()
wandb.finish()

#### Testing the training

In [ ]:
# evaluation
predictions = trainer.predict(dataset['test'])

In [ ]:
# saving model weigths into files
model.save_pretrained('./saved_model/')

a. Discuss at least four relevant hyper-parameters

Learning rate : \
Epoch Numer : \
Batch Size : \
Optimizer: \
Layer Number : [??]

In [ ]:
# learning rate
# epoch number
# mini-batch size

b. Experiment with the effect of different batch sizes

c. Experiment with the effect of different learning rates

d. Experiment with different number of network layers

e. Implement at least two data agumentation techniques

f. Discuss what influences the memory use of a solution such as yours. What can be done to reduce this?